# Prefect

``rubicon_ml`` offers an integration with [Prefect](https://www.prefect.io/), an open source
workflow management engine used heavily within the Python ecosystem. In Prefect, a unit of
work is called a **task**, and a collection of **tasks** makes up a **flow**. A **flow**
represents your workflow pipeline. You can integrate ``rubicon_ml`` into your workflows to
persist metadata about your experimentation.

We'll [run a Prefect server locally](https://docs.prefect.io/core/getting_started/installation.html#running-the-local-server-and-ui)
for this example. If you already have Prefect and Docker installed, you can start a
Prefect server and agent with these commands:

```bash
prefect backend server
prefect server start

# and in a new terminal
prefect agent start
```

For more context, check out the
[workflow README on GitHub](https://github.com/capitalone/rubicon-ml/tree/main/rubicon/workflow).

### Setting up a simple flow

Now we can get started! Creating Prefect **tasks** is easy enough on its own, but we've added
some simple ones to the ``rubicon_ml`` library.

In [ ]:
from rubicon_ml.workflow.prefect import (
    get_or_create_project_task,
    create_experiment_task,
    log_artifact_task,
    log_dataframe_task,
    log_feature_task,
    log_metric_task,
    log_parameter_task,
)

We'll need a workflow to integrate ``rubicon_ml`` logging into, so let's put together a simple one.
To mimic a realistic example, we'll create tasks for loading data, splitting said data, extracting
features, and training a model.

In [ ]:
from prefect import task


@task
def load_data():
    from sklearn.datasets import load_wine
    
    return load_wine()

In [ ]:
@task
def split_data(dataset):
    from sklearn.model_selection import train_test_split
    
    
    return train_test_split(
        dataset.data,
        dataset.target,
        test_size=0.25,
    )

In [ ]:
@task
def get_feature_names(dataset):
    return dataset.feature_names

In [ ]:
@task
def fit_pred_model(
    train_test_split_result,
    n_components,
    n_neighbors,
    is_standardized
):
    from sklearn import metrics
    from sklearn.decomposition import PCA
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.pipeline import make_pipeline
    from sklearn.preprocessing import StandardScaler

    
    X_train, X_test, y_train, y_test = train_test_split_result

    if is_standardized:
        classifier = make_pipeline(
            StandardScaler(),
            PCA(n_components=n_components),
            KNeighborsClassifier(n_neighbors=n_neighbors),
        )
    else:
        classifier = make_pipeline(
            PCA(n_components=n_components),
            KNeighborsClassifier(n_neighbors=n_neighbors),
        )
        
    classifier.fit(X_train, y_train)
    pred_test = classifier.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, pred_test)
    
    return accuracy

Without ``rubicon_ml``, here's what this simple **flow** would look like:

In [ ]:
from prefect import Flow


n_components = 2
n_neighbors = 5
is_standardized = True

with Flow("Wine Classification") as flow:
    wine_dataset = load_data()
    
    feature_names = get_feature_names(wine_dataset)
    train_test_split = split_data(wine_dataset)
    
    accuracy = fit_pred_model(
        train_test_split,
        n_components,
        n_neighbors,
        is_standardized,
    )

### Running a flow and viewing results

Now we'll register our **flow** with the local server. First, we'll have to
use the ``prefect`` CLI one more time to create a project.

```bash
prefect create project 'Wine Classification'
```

The URL printed from the call to ``flow.register`` opens the local Prefect UI.
We can use it to run and monitor our **flows**. 

In [ ]:
flow_id = flow.register(project_name="Wine Classification")

We'll also put together a function to run our **flows** and wait for them to finish.

In [ ]:
import time
from prefect import Client


prefect_client = Client()

def run_flow(client, flow_id):
    flow_run_id = client.create_flow_run(flow_id=flow_id)
    
    is_finished = False
    while not is_finished:
        state = client.get_flow_run_info(flow_run_id).state
        print(f"{state.message.strip('.')}. Waiting...")
        
        time.sleep(3)
        
        is_finished = state.is_finished()

    assert state.is_successful()
    print(f"Flow run {flow_run_id} was successful!")
    
    return flow_run_id
    
flow_run_id = run_flow(prefect_client, flow_id)

We assigned a few variables in our **flow**, most notably the result of ``fit_pred_model``, ``accuracy``.
This accuracy metric is how we'll determine whether or not the model we trained is a success. However,
retrieving state variables from **flows** is a bit cumbersome.

In [ ]:
info = prefect_client.get_flow_run_info(flow_run_id)

slugs = [t.task_slug for t in info.task_runs]
index = slugs.index(accuracy.slug)

result = info.task_runs[index].state._result.read(
    info.task_runs[index].state._result.location,
)
result.value

What's going on here isn't exactly intuitive, and all that only extracted one
piece of data from one task.
 
### Adding Rubicon to your flow

We can leverage the Prefect tasks within the ``rubicon_ml`` library to log all the
info we want about our model run. Then, we can use the standard ``rubicon_ml`` logging
library to retrieve and inspect our metrics and other logged data. This is much
simpler than digging through the state of each executed **task** and extracting
its results.

Here's the same flow from above, this time with ``rubicon_ml`` **tasks** integrated.

In [ ]:
import os

from prefect import unmapped


root_path = f"{os.getcwd()}/rubicon-root"

n_components = 2
n_neighbors = 5
is_standardized = True

with Flow("Wine Classification with Rubicon") as flow:    
    project = get_or_create_project_task(
        "filesystem",
        root_path,
        "Wine Classification with Prefect",
    )
    experiment = create_experiment_task(
        project,
        name="logged from a Prefect task",
    )
    
    wine_dataset = load_data()
    feature_names = get_feature_names(wine_dataset)
    train_test_split = split_data(wine_dataset)
    
    log_feature_task.map(unmapped(experiment), feature_names)
    log_parameter_task(experiment, "n_components", n_components)
    log_parameter_task(experiment, "n_neighbors", n_neighbors)
    log_parameter_task(experiment, "is_standardized", is_standardized)
    
    accuracy = fit_pred_model(
        train_test_split,
        n_components,
        n_neighbors,
        is_standardized,
    )
    
    log_metric_task(experiment, "accuracy", accuracy)

Again, we'll register and run the **flow**.

In [ ]:
flow_with_rubicon_id = flow.register(project_name="Wine Classification")
flow_run_with_rubicon_id = run_flow(prefect_client, flow_with_rubicon_id)

This time we can use ``rubicon_ml`` to inspect our accuracy, among other things!

In [ ]:
from rubicon_ml import Rubicon


rubicon = Rubicon(persistence="filesystem", root_dir=root_path)
project = rubicon.get_project("Wine Classification with Prefect")
    
experiment = project.experiments()[0]

features = [f.name for f in experiment.features()]
parameters = [(p.name, p.value) for p in experiment.parameters()]
metrics = [(m.name, m.value) for m in experiment.metrics()]

print(
    f"experiment {experiment.id}\n"
    f"features: {features}\nparameters: {parameters}\n"
    f"metrics: {metrics}"
)

dask_client.cluster.scheduler.addressExtracting data from ``rubicon_ml`` is much simpler than pulling it from the various
**tasks** as they succeed. Especially in the case where we're running thousands
of **tasks** in a **flow**.

### Concurrent Logging with Dask

Prefect plays nicely with Dask in order to provide parallel computing when possible.
The Prefect scheduler is smart enough to know which **tasks** do and do not depend
on each other, so it can intelligently decide when to run independent **tasks** at
the same time.

Let's run the same **flow** as above, except this time we'll log eight different
experiments with eight different feature sets and accuracy results. First, we'll
need to use Dask to start a local cluster.

In [ ]:
import dask.distributed
from prefect.executors import DaskExecutor


dask_client = dask.distributed.Client()
dask_client

In [ ]:
dask_executor = DaskExecutor(address=dask_client.cluster.scheduler.address)

Before we look at the new **flow**, let's see how easy it is to make our own 
``rubicon_ml`` Prefect **tasks**. Currently, the ``log_feature_task`` logs one 
feature to one experiment. Let's say in this example, we want to log our entire
feature set in one **task**. That's slightly different than what we currently
have in the ``log_feature_task``, so let's see how we can make a new one using
the ``rubicon_ml`` logging library.

In [ ]:
@task
def log_feature_set(experiment, feature_names):
    """log a set of features to a rubicon experiment
    
    Parameters
    ----------
    experiment : rubicon.Experiment
        the experiment to log the feature set to
    feature_names : list of str
        the names of the features to log to `experiment`
    """
    features = []
    
    for name in feature_names:
        features.append(experiment.log_feature(name=name))
        
    return features

Easy! Even though its so simple, this particular **task** is actually more in-depth
than the ones you'll find in the library. The Prefect **tasks** in the library are
simply wrappers around existing logging library functions, like ``experiment.log_feature``.

Now we can make our new **flow** to log multiple experiments in parallel.

In [ ]:
n_components =    [2,    2,     2,    2,     4,    4,     4,    4    ]
n_neighbors =     [5,    5,     10,   10,    5,    5,     10,   10   ]
is_standardized = [True, False, True, False, True, False, True, False]

experiment_names = [f"mapped run {i}" for i in range(len(n_components))]

with Flow(
    "Wine Classification with Rubicon - Mapped",
    executor=dask_executor,
) as mapped_flow:
    project = get_or_create_project_task(
        "filesystem",
        root_path,
        "Wine Classification with Prefect - Mapped",
    )
    experiments = create_experiment_task.map(
        unmapped(project),
        name=experiment_names,
        description=unmapped("concurrent example with Prefect"),
    )
    
    wine_dataset = load_data()
    
    feature_names = get_feature_names(wine_dataset)
    train_test_split = split_data(wine_dataset)
    
    log_feature_set.map(experiments, unmapped(feature_names))
    log_parameter_task.map(experiments, unmapped("n_components"), n_components)
    log_parameter_task.map(experiments, unmapped("n_neighbors"), n_neighbors)
    log_parameter_task.map(experiments, unmapped("is_standardized"), is_standardized)
    
    accuracies = fit_pred_model.map(
        unmapped(train_test_split),
        n_components,
        n_neighbors,
        is_standardized,
    )
 
    log_metric_task.map(experiments, unmapped("accuracy"), accuracies)

Let's register and run one last **flow**. If you check out the timeline for your completed
**flows** on the UI linked by `mapped_flow.register`, you'll notice tasks executing at the same
time now.

In [ ]:
flow_with_concurrent_rubicon_id = mapped_flow.register(
    project_name="Wine Classification",
)
flow_run_with_concurrent_rubicon_id = run_flow(
    prefect_client,
    flow_with_concurrent_rubicon_id,
)

Retrieving all the results of a mapped **task** is even more cumbersome than
retrieving just the accuracy above. We'll simply use the ``rubicon_ml`` Dashboard to
check out all the data we just logged!

In [ ]:
from rubicon_ml.ui import Dashboard


Dashboard(persistence="filesystem", root_dir=root_path).run_server()

### Cleanup

In [ ]:
dask_client.shutdown()